# CryoEM Example - Synthetic and EMPIAR

In [ ]:
import scivision
import intake

In [ ]:
# TODO: make odin an installable package
model = scivision.load_pretrained_model("https://github.com/alan-turing-institute/odin")

## Synthetic data example

Data intake catalog (goes elsewhere)

In [ ]:
%%writefile scivision_soup_data.yml
sources:
    synthetic_soup:
        description: Generate numpy array of synthetic cryoEM images
        driver: synthetic_alphabetsoup
        args:
            datashape: [500, 500]
            image_count: 10
            ctf_defocus: 5e3
            ctf_box_size: 512
            # seed: 1234

In [ ]:
cat_scivision = scivision.load_dataset('scivision_soup_data.yml')

In [ ]:
# Alternatively: load directly from intake

# A synthetic cryoEM dataset
cat_intake = intake.open_synthetic_alphabetsoup()

In [ ]:
# a numpy array of shape (1000,500,500,3)
# TODO:
# images, bounding_boxes, labels = cat_scivision.synthetic_soup.read()

images = cat_scivision.synthetic_soup.read()

In [ ]:
# bounding boxes are
#  np.array([x_top_left, y_top_left, x_bottom_right, y_bottom_right])

In [ ]:
# array of bounding boxes, shape (Nboxes, 4) - no labels required
model.predict(images[0])

## EMPIAR example

Data intake catalog (goes elsewhere)

In [ ]:
%%writefile scivision_empiar_data.yml
sources:
    one_empiar_dataset:
        driver: empiar_driver
            args:
                empiar_index: 10185
                mrc_directory: "directory" # optional
                star_directory: "directory" # optional

# perhaps: xarray DataSet with mrc array (ragged) and star array (?) - currently separate arrays

In [ ]:
cat2 = scivision.load_dataset('scivision_empiar_data.yml')

In [ ]:
bbs_empiar = [model.predict(img) for img in images]